# Pilvilaskenta ja data-analytiikka: "Azure ML SDK" -lähestymistapa

## Johdanto

Tässä notebookissa opimme käyttämään Azure ML SDK:ta mallin kouluttamiseen, käyttöönottoon ja kuluttamiseen Azure ML:n kautta.

Esivaatimukset:
1. Olet luonut Azure ML -työtilan.
2. Olet ladannut [Heart Failure -datan](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) Azure ML:ään.
3. Olet ladannut tämän notebookin Azure ML Studioon.

Seuraavat vaiheet ovat:

1. Luo kokeilu olemassa olevaan työtilaan.
2. Luo laskentaklusteri.
3. Lataa datasetti.
4. Määritä AutoML käyttäen AutoMLConfigia.
5. Suorita AutoML-kokeilu.
6. Tutki tuloksia ja valitse paras malli.
7. Rekisteröi paras malli.
8. Ota paras malli käyttöön.
9. Käytä päätepistettä.

## Azure Machine Learning SDK -spesifiset tuonnit


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Alusta Työtila
Alusta työtilaobjekti tallennetusta konfiguraatiosta. Varmista, että konfiguraatiotiedosto on läsnä polussa .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Luo Azure ML -kokeilu

Luodaan kokeilu nimeltä 'aml-experiment' juuri alustettuun työtilaan.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Luo laskentaklusteri  
Sinun täytyy luoda [laskentakohde](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) AutoML-ajollesi.  


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Data
Varmista, että olet ladannut tietoaineiston Azure ML:ään ja että avain on sama kuin tietoaineiston nimi.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## AutoML-konfiguraatio


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## AutoML ajo


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Ota paras malli käyttöön

Suorita seuraava koodi ottaaksesi parhaan mallin käyttöön. Voit tarkastella käyttöönoton tilaa Azure ML -portaalissa. Tämä vaihe voi kestää muutaman minuutin.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Käytä päätepistettä
Voit lisätä syötteitä seuraavaan syöte-esimerkkiin.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäinen asiakirja sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
